In [1]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib

os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU

utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [2]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 2e-3
# increase epochs for better results
epochs = 100
sample_freq = 1

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

# fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channels, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}_sqa'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=True, nrow=10)
        tv.utils.save_image(model.unpatchify(z[:100]), sample_dir / f'latent_{epoch:03d}.png', normalize=True, nrow=10)
        print('sampling complete')
torch.save(model.state_dict(), ckpt_file)

using device cuda
Number of parameters: 3.40M
epoch 0 lr 0.002000 loss -1.4405
layer norms 0.5788 1.1133 1.4094 1.6728
epoch 1 lr 0.001999 loss -2.2376
layer norms 0.8418 2.1435 4.4628 1.6956
sampling complete
epoch 2 lr 0.001998 loss -2.9081
layer norms 0.8446 3.0001 5.4668 0.7784
epoch 3 lr 0.001995 loss -3.1655
layer norms 0.5348 3.2221 7.5155 1.1686
sampling complete
epoch 4 lr 0.001992 loss -3.2507
layer norms 0.4766 3.1361 6.6913 0.7953
epoch 5 lr 0.001987 loss -3.3728
layer norms 0.4533 2.9274 5.0892 0.9770
sampling complete
epoch 6 lr 0.001982 loss -3.4808
layer norms 0.4213 3.8369 7.0853 1.0374
epoch 7 lr 0.001975 loss -3.5555
layer norms 0.3671 2.3712 3.6965 0.6893
sampling complete
epoch 8 lr 0.001968 loss -3.3186
layer norms 0.3939 3.0440 3.5191 0.8767
epoch 9 lr 0.001960 loss nan
layer norms nan nan nan nan


/afs/csail.mit.edu/u/s/sqa24/work/tarflow-sqa/transformer_flow.py:328: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at ../aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:697.)
  x = torch.matmul(x - bias, torch.linalg.pinv(weight).T)


_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 16).

In [ ]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')

Accuracy %98.35
